In [1]:
import re
import os
import pickle
import numpy as np
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate
from data.configs.demo import config
from torch.utils.data import DataLoader

In [2]:
from utils.MIND import MIND_bert

In [11]:
config.embedding = 'bert'
config.signal_length = 20
config.his_size = 10
train_path = config.path + 'MIND/MINDdemo_dev/'
a = MIND_bert(config, train_path + 'news.tsv', train_path + 'behaviors.tsv')

In [5]:
import pickle

imprs = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/bert/MINDdemo_dev/behaviors.pkl', 'rb'))['imprs']

In [7]:
for i in imprs:
    print(i[1])

[36180, 41328, 41034, 39776, 34983, 37322, 37327, 36307, 36185, 36349, 38581, 39227, 37368, 33705, 39921, 39640, 36275, 38848, 7014, 37943, 39086, 24209]
[35201, 40369, 37645, 36756, 39488, 39669, 41034]
[39669, 38581, 33705, 37322, 37739, 39201, 36185, 38092, 35201, 36022, 35148, 41034, 37327, 39227, 35760, 39640, 167, 7014, 37210, 39921, 38848, 34983, 39086]
[438, 37333, 4393, 7014, 34983, 36307, 41328, 13426, 37444, 36275, 38581, 39500, 38139, 39640, 38865, 37368, 36781, 39450, 24209, 36185, 36411, 39483, 39776, 39762, 38848, 38407]
[39227, 40369, 39640, 35825, 36185, 41034, 35882, 167, 36781, 38581, 36275, 39776, 39669, 35201]
[24209, 41328, 36499, 37322, 38848, 36307, 37368, 38865, 36781, 39086, 41034]
[38496, 167, 39669, 35201, 37645, 39227, 41034]
[38417, 37947, 38320, 40247, 39559, 41102, 42123, 37361, 38174, 40033, 38892, 41116, 40378, 35822, 41275, 40672, 41356, 36162, 39661, 40254, 40078, 40821, 41005, 36077, 36861, 39227, 38324, 41302, 41718, 39003, 40000, 40183, 39440, 385

In [7]:
from torch.utils.data.distributed import DistributedSampler
s = DistributedSampler(a, num_replicas=2, rank=0, shuffle=False)

In [18]:
loader_train = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, sampler=None)
record = next(iter(loader_train))

In [16]:
record

{'impression_index': [1],
 'user_index': tensor([[1929]]),
 'cdd_id': tensor([[36180, 41328, 41034, 39776, 34983, 37322, 37327, 36307, 36185, 36349,
          38581, 39227, 37368, 33705, 39921, 39640, 36275, 38848,  7014, 37943,
          39086, 24209]]),
 'his_id': tensor([[ 7499, 34171, 15796, 32731, 32139, 18228,  1736, 30270, 35479, 30833]]),
 'cdd_encoded_index': tensor([[[13240, 12134,  2000,  6701, 18466,  1010,  2655, 27056,  9674,  1005,
            1055,  4506,  1005, 21873,  1005,  2867,  2411,  6985,  2037, 13220],
          [ 1045,  1005,  2310,  2042,  3015,  2055,  4714,  5014,  2005,  1037,
            2095,  1998,  2633,  2985,  1016,  6385,  1999,  1037,  3998,  1011],
          [ 5448,  1024,  6972, 10556, 13699, 11795,  6799,  2003,  2055,  2000,
            2131,  2054,  2002, 17210,  1024,  1037,  3382,  1996,  2203,  2089],
          [ 1996, 10556, 13639,  6182,  6962,  2227, 25748,  2058,  1005, 16021,
            6132, 13043,  1005,  2155,  2833,  2954,  1999, 

In [8]:
loader_train = DataLoader(b, batch_size=config.batch_size, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, collate_fn=my_collate, sampler=None)
record = next(iter(loader_train))

In [11]:
record['cdd_attn_mask'].shape, record['cdd_encoded_index'].shape

(torch.Size([10, 5, 20]), torch.Size([10, 5, 20]))